In [ ]:
import imageio.v3 as iio
import cv2
from matplotlib import pyplot as plt
from IPython.display import Image, Video
from openface.face_detection import FaceDetector

# The next block fixes a bug that causes crashes on Unix-based systems.
# You should apply this patch before importing "LandmarkDetector" from Openface.

# --- begin of patch ---
# Patching LandmarkDetector hardcoded log folder
from openface.STAR.demo import Alignment

# Access the utility module through Alignment.__init__'s globals
utility = Alignment.__init__.__globals__['utility']

# Only patch once to avoid recursion
if getattr(utility.set_environment, '__name__', '') != 'patched_set_environment':
    original_set_environment = utility.set_environment

    def patched_set_environment(config):
        result = original_set_environment(config)
        config.log_dir = '.'
        return result

    utility.set_environment = patched_set_environment
# --- end of patch ---

from openface.landmark_detection import LandmarkDetector
from openface.multitask_model import MultitaskPredictor
import torch.nn.functional as F
import torch
import numpy as np
from typing import Union
import os


# The next patch overrides a function in FaceDetector which expects a filename as input to also accept numpy arrays
# This will be useful later when we want to pass webcam frames directly without saving to disk

# --- begin of patch ---
def override_preprocess_image(self, image_path: Union[str, np.ndarray], resize: float = 1.0):
    if isinstance(image_path, (str, os.PathLike)):
        img_raw = cv2.imread(str(image_path), cv2.IMREAD_COLOR)  # BGR, 3 channels
        if img_raw is None:
            raise ValueError(f"Failed to read image from path: {image_path}")
    elif isinstance(image_path, np.ndarray):
        img_raw = image_path
    else:
        raise TypeError("image_path must be a str/Path-like path or a numpy.ndarray (BGR frame).")

    if img_raw.ndim == 2:
        # Grayscale -> BGR
        img_raw = cv2.cvtColor(img_raw, cv2.COLOR_GRAY2BGR)
    elif img_raw.ndim == 3:
        if img_raw.shape[2] == 4:
            # BGRA -> BGR
            img_raw = cv2.cvtColor(img_raw, cv2.COLOR_BGRA2BGR)
        elif img_raw.shape[2] != 3:
            raise ValueError(f"Unsupported channel count: {img_raw.shape[2]} (expected 1, 3, or 4)")
    else:
        raise ValueError(f"Unsupported image shape {img_raw.shape}; expected HxW or HxWxC.")

    # --- Preprocess as in original code
    img = img_raw.astype(np.float32, copy=False)
    if resize != 1.0:
        img = cv2.resize(img, None, fx=resize, fy=resize, interpolation=cv2.INTER_LINEAR)

    # Mean subtraction in BGR (matching many Caffe-style models)
    img -= (104.0, 117.0, 123.0)

    # Ensure contiguous before transpose (safer with slices or unusual strides)
    img = np.ascontiguousarray(img.transpose(2, 0, 1))  # (C, H, W)

    img = torch.from_numpy(img).unsqueeze(0).to(self.device)  # (1, C, H, W)

    return img, img_raw

FaceDetector.preprocess_image = override_preprocess_image
# --- end of patch ---

c:\NTU\Uppsala\Intelligent Agents\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from sklearn.model_selection import train_test_split  # for splitting data if needed
from sklearn.metrics import accuracy_score, classification_report  # for evaluation
from pathlib import Path
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from PIL import Image
from tqdm import tqdm
import joblib



# using this to quickly load a random dataset. For the assignment and this lab, you will be provided with a specific dataset.
# NOTE that this your current environment *does not have* this library as it is not required to do the project/assignment
from datasets import load_dataset

In [6]:
device = "cpu"  # or "cpu"

det = FaceDetector("./weights/Alignment_RetinaFace.pth", device=device)
lmk = LandmarkDetector("./weights/Landmark_98.pkl", device=device)

# helper function to get landmarks in order to train a landmark->emotion model
def detect_face_and_get_landmarks(image):
    # transform PIL image to BGR numpy array
    img_bgr = cv2.cvtColor(np.array(image.convert("RGB")), cv2.COLOR_RGB2BGR)

    _, dets = det.get_face(img_bgr)
    if dets is None or len(dets) == 0:
        image.save("no_face_detected.png")
        return None

    pts = lmk.detect_landmarks(img_bgr, [dets[0]])[0].astype(np.float32)  # (98,2)
    return pts

2026-01-09 00:03:48,178 INFO    : Loaded configure file alignment: 3483e3d0-84d2-42b2-8812-93b32730fb0f
2026-01-09 00:03:48,178 INFO    : Loaded configure file alignment: 3483e3d0-84d2-42b2-8812-93b32730fb0f
2026-01-09 00:03:48,180 INFO    : 
type: alignment
id: 3483e3d0-84d2-42b2-8812-93b32730fb0f
note: 
ckpt_dir: /work/jiewenh/openFace/OpenFace-3.0/STAR
image_dir: ./WFLW\WFLW_images
annot_dir: ./
loader_type: alignment
loss_func: STARLoss_v2
batch_size: 32
val_batch_size: 32
test_batch_size: 16
channels: 3
width: 256
height: 256
means: (127.5, 127.5, 127.5)
scale: 0.00784313725490196
display_iteration: 10
milestones: [200, 350, 450]
max_epoch: 500
net: stackedHGnet_v1
nstack: 4
optimizer: adam
learn_rate: 0.001
momentum: 0.01
weight_decay: 1e-05
nesterov: False
scheduler: MultiStepLR
gamma: 0.1
loss_weights: [0.125, 1.25, 1.25, 0.25, 2.5, 2.5, 0.5, 5.0, 5.0, 1.0, 10.0, 10.0]
criterions: ['STARLoss_v2', 'AWingLoss', 'AWingLoss', 'STARLoss_v2', 'AWingLoss', 'AWingLoss', 'STARLoss_v2', 

Loading pretrained model from ./weights/Alignment_RetinaFace.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Namespace(config_name='alignment', device_id='cpu')


In [9]:
from pathlib import Path
from sklearn.model_selection import train_test_split

# ----------------------------
# Config
# ----------------------------
# Path to your dataset (relative from your notebook location)
dataset_dir = Path("../DiffusionFER/DiffusionFER/DiffusionEmotion_S/cropped")

multitask_model_path = "./weights/MTL_backbone.pth"
emo_names = ["neutral", "happy", "sad", "surprise", "fear", "disgust", "anger", "contempt"]
device = "cuda" if torch.cuda.is_available() else "cpu"

dataset_emo_mapping = sorted([f.name for f in dataset_dir.iterdir() if f.is_dir()])
print(f"Detected classes: {dataset_emo_mapping}")

# Initialize MTL
mtl = MultitaskPredictor(model_path=multitask_model_path, device=device)

# Map MTL outputs to dataset labels
mtl_to_dataset = {name: name for name in dataset_emo_mapping}
mtl_to_dataset["contempt"] = None  # ignore contempt

Detected classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Loading multitask model from ./weights/MTL_backbone.pth...


c:\NTU\Uppsala\Intelligent Agents\.venv\Lib\site-packages\timm\models\_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b0_ns to current tf_efficientnet_b0.ns_jft_in1k.
  model = create_fn(


In [22]:
from sklearn.model_selection import train_test_split

# Load ALL data first
all_images, all_labels = [], []
for emo_folder in dataset_dir.iterdir():
    if not emo_folder.is_dir():
        continue
    label_name = emo_folder.name
    if label_name not in dataset_emo_mapping:
        continue
    
    # Handle nested structure: angry/angry/*.png
    nested_folder = emo_folder / label_name
    if nested_folder.exists() and nested_folder.is_dir():
        img_folder = nested_folder
    else:
        img_folder = emo_folder
    
    for img_path in img_folder.glob("*.png"):
        all_images.append(Image.open(img_path).convert("RGB"))
        all_labels.append(label_name)

print(f"Loaded {len(all_images)} total images")

# Split into train and test (80/20 split)
train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

print(f"Train: {len(train_images)} images, Test: {len(test_images)} images")

# Encode labels
le = LabelEncoder()
train_labels_encoded = le.fit_transform(train_labels)

# Extract features
train_features = extract_mtl_features(train_images)
train_dataset = EmotionDataset(train_features, train_labels_encoded)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

2026-01-09 01:01:44,563 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 01:01:44,563 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 01:01:44,565 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 01:01:44,565 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 01:01:44,567 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 01:01:44,567 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 01:01:44,568 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 01:01:44,568 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 01:01:44,570 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 01:01:44,570 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 01:01:44,572 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 01:01:44,572 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 01:01:44,573 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 01:01:44,573 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 01:01:44,576 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 01:01:44,576 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 01:01:44,579 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 01:01:44,579 DEBUG   : STREAM 

Loaded 1289 total images
Train: 1031 images, Test: 258 images


Extracting MTL features: 100%|██████████| 1031/1031 [00:44<00:00, 23.29it/s]


In [10]:
# Helper functions
def get_image_emotion(image):
    img_bgr = cv2.cvtColor(np.array(image.convert("RGB")), cv2.COLOR_RGB2BGR)
    cropped_face, dets = det.get_face(img_bgr)
    if cropped_face is None or dets is None or len(dets) == 0 or cropped_face.size == 0:
        return None
    emotion_logits, _, _ = mtl.predict(cropped_face)
    probs = F.softmax(emotion_logits, dim=1)[0].cpu().numpy()
    emo_idx = int(np.argmax(probs))
    return emo_names[emo_idx]

def extract_mtl_features(images):
    """Convert MTL predictions to one-hot features"""
    features = []
    for img in tqdm(images, desc="Extracting MTL features"):
        emo_pred = get_image_emotion(img)
        if emo_pred is None:
            features.append(np.zeros(len(emo_names)))
            continue
        one_hot = np.zeros(len(emo_names))
        one_hot[emo_names.index(emo_pred)] = 1
        features.append(one_hot)
    return np.array(features, dtype=np.float32)


In [11]:
# Dataset class
class EmotionDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return torch.tensor(self.images[idx]), torch.tensor(self.labels[idx])


In [9]:
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image

dataset_dir = Path("../DiffusionFER/DiffusionFER/DiffusionEmotion_S/cropped")
dataset_emo_mapping = sorted([f.name for f in dataset_dir.iterdir() if f.is_dir()])

device = "cuda" if torch.cuda.is_available() else "cpu"

det = FaceDetector("./weights/Alignment_RetinaFace.pth", device=device)
lmk = LandmarkDetector("./weights/Landmark_98.pkl", device=device)

all_images, all_labels = [], []
for emo_folder in dataset_dir.iterdir():
    if not emo_folder.is_dir():
        continue
    label_name = emo_folder.name
    if label_name not in dataset_emo_mapping:
        continue
    
    nested_folder = emo_folder / label_name
    if nested_folder.exists() and nested_folder.is_dir():
        img_folder = nested_folder
    else:
        img_folder = emo_folder
    
    for img_path in img_folder.glob("*.png"):
        all_images.append(Image.open(img_path).convert("RGB"))
        all_labels.append(label_name)

print(f"Loaded {len(all_images)} total images")

train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42, stratify=all_labels
)

print(f"Train: {len(train_images)}, Test: {len(test_images)}")

le = LabelEncoder()
le.fit(all_labels)
train_labels_encoded = le.transform(train_labels)

c:\NTU\Uppsala\Intelligent Agents\.venv\Lib\site-packages\torch\__init__.py:1275: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\tensor\python_tensor.cpp:436.)
  _C._set_default_tensor_type(t)
2026-01-09 21:15:42,242 INFO    : Loaded configure file alignment: 47736574-e532-4b8b-8f46-b73fee28ba98
2026-01-09 21:15:42,244 INFO    : 
type: alignment
id: 47736574-e532-4b8b-8f46-b73fee28ba98
note: 
ckpt_dir: /work/jiewenh/openFace/OpenFace-3.0/STAR
image_dir: ./WFLW\WFLW_images
annot_dir: ./
loader_type: alignment
loss_func: STARLoss_v2
batch_size: 32
val_batch_size: 32
test_batch_size: 16
channels: 3
width: 256
height: 256
means: (127.5, 127.5, 127.5)
scale: 0.00784313725490196
display_iteration: 10
milestones: [200, 350, 450]
max_epoch: 500
net: stackedHGnet_v1
nstack: 4
optimizer: adam
learn_

Loading pretrained model from ./weights/Alignment_RetinaFace.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Namespace(config_name='alignment', device_id='cpu')


2026-01-09 21:15:44,030 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 21:15:44,032 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 21:15:44,037 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 21:15:44,038 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 21:15:44,042 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 21:15:44,043 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 21:15:44,047 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 21:15:44,047 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 21:15:44,055 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 21:15:44,057 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 21:15:44,062 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 21:15:44,062 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 21:15:44,069 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 21:15:44,069 DEBUG   : STREAM b'IDAT' 41 65536
2026-01-09 21:15:44,076 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 21:15:44,083 DEBUG   : STREAM b'IDAT' 41 64989
2026-01-09 21:15:44,099 DEBUG   : STREAM b'IHDR' 16 13
2026-01-09 21:15:44,128 DEBUG   : STREAM 

Loaded 1289 total images
Train: 1031, Test: 258


In [10]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import joblib

def extract_landmarks(image):
    try:
        img_bgr = cv2.cvtColor(np.array(image.convert("RGB")), cv2.COLOR_RGB2BGR)
        _, dets = det.get_face(img_bgr)
        if dets is None or len(dets) == 0:
            return None
        pts = lmk.detect_landmarks(img_bgr, [dets[0]])[0].astype(np.float32)
        return pts.flatten()
    except:
        return None

train_landmarks = []
train_labels_valid = []
skipped_train = 0

for img, label in zip(train_images, train_labels_encoded):
    landmarks = extract_landmarks(img)
    if landmarks is not None:
        train_landmarks.append(landmarks)
        train_labels_valid.append(label)
    else:
        skipped_train += 1

test_landmarks = []
test_labels_valid = []
skipped_test = 0

for img, label in zip(test_images, le.transform(test_labels)):
    landmarks = extract_landmarks(img)
    if landmarks is not None:
        test_landmarks.append(landmarks)
        test_labels_valid.append(label)
    else:
        skipped_test += 1

X_train = np.array(train_landmarks)
X_test = np.array(test_landmarks)
y_train_svm = np.array(train_labels_valid)
y_test_svm = np.array(test_labels_valid)

print(f"Train: {len(X_train)} samples (skipped {skipped_train})")
print(f"Test: {len(X_test)} samples (skipped {skipped_test})")

Processing face: (13, 13, 206, 202), confidence: 0.9992730021476746


c:\NTU\Uppsala\Intelligent Agents\.venv\Lib\site-packages\torch\functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4319.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Processing face: (17, 11, 200, 206), confidence: 0.9994262456893921
Processing face: (15, 14, 206, 216), confidence: 0.9995145797729492
Processing face: (14, 11, 202, 204), confidence: 0.9994766116142273
Processing face: (19, 20, 215, 216), confidence: 0.9995852112770081
Processing face: (18, 9, 207, 227), confidence: 0.9992846846580505
Processing face: (16, 11, 197, 217), confidence: 0.9993304014205933
Processing face: (15, 16, 215, 216), confidence: 0.9996602535247803
Processing face: (15, 14, 203, 214), confidence: 0.999458372592926
Processing face: (20, 12, 207, 210), confidence: 0.9994361996650696
Processing face: (17, 19, 208, 208), confidence: 0.9992998838424683
Processing face: (16, 18, 210, 215), confidence: 0.9996589422225952
Processing face: (31, 18, 209, 217), confidence: 0.9983803033828735
Processing face: (29, 8, 214, 204), confidence: 0.9992901086807251
Processing face: (33, 12, 203, 209), confidence: 0.9979782700538635
Processing face: (14, 9, 203, 205), confidence: 0.9

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_model.fit(X_train_scaled, y_train_svm)

joblib.dump(svm_model, "emotion_svm_landmarks.pkl")
joblib.dump(scaler, "scaler_landmarks.pkl")
joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']

In [12]:
svm_model = joblib.load("emotion_svm_landmarks.pkl")
scaler = joblib.load("scaler_landmarks.pkl")
le = joblib.load("label_encoder.pkl")

y_pred = svm_model.predict(X_test_scaled)
test_labels_pred = le.inverse_transform(y_pred)
test_labels_true = le.inverse_transform(y_test_svm)

print(f"Test Accuracy: {accuracy_score(test_labels_true, test_labels_pred)*100:.2f}%")
print(classification_report(test_labels_true, test_labels_pred, zero_division=0))

Test Accuracy: 66.54%
              precision    recall  f1-score   support

       angry       0.56      0.40      0.47        35
     disgust       0.00      0.00      0.00        18
        fear       0.75      0.21      0.33        14
       happy       0.98      0.90      0.94        68
     neutral       0.54      0.99      0.70        70
         sad       0.00      0.00      0.00        16
    surprise       0.65      0.67      0.66        36

    accuracy                           0.67       257
   macro avg       0.50      0.45      0.44       257
weighted avg       0.62      0.67      0.61       257

